<a href="https://colab.research.google.com/github/wizard339/education/blob/main/tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install tokenizers

# 🤗 Tokenizers

Для работы с текстом нам нужно уметь представлять его в понятном копмьютеру виде и первые шаги в направлении - это разбить текст на токены (которые могут быть словами, а могут и не быть в зависимости от метода токенизации) и сопоставить каждому слову уникальный номер от 0 до размера вашего словаря. После этого мы сможем конвертировать строку типа "это токенизированная строка" в вектор вида `[47, 392, 38]`. Эта операция - замены токенов на их индексы - часто называется **нумерикализация**, а обратная операция - **денумерикализация**.

Если вы используете модуль `sklearn.feature_extraction.text`, например для векторизации с помощью `TfidfVectorizer`, sklearn выполняет все эти операции за вас под капотом, но если вы хотите использовать нейросети, жизнь становится гораздо сложнее и до недавнего времени почти все использовали самописные штуки, чтобы (де)нумерикализовать тексты. Это создавало множество неудобст связанных с тем, что во-первых вам приходилось писать больше кода (и создавать больше ошибок), во-вторых рядом с нумерикализацией лежит очень много мелких подзадач включая использование специальных токенов, таких как UNK, сериализация ваших объектов, и распараллеливание на несколько потоков для ускорения. 

Huggingface Tokenizers постепенно становятся всё более и более популярными в NLP-сообществе. Эта библиотека быстро развивается и позволяет вам использовать различные методы токенизации - от разделения на слова по пробелу до разделения на битовые представления unicode-строк в зависимости от частоты битовой n-gram. Также она умеет делать нумерикализацию и очень быстро работает. Вместо самописных подходов, которые всё ещё часто преподаются в курсах по NLP, мы будем использовать 🤗 Tokenizers в течение всего курса и довольно быстро вы научитесь пользоваться даже не самыми очевидными её методами. А сейчас мы начнём с самых основ.

In [2]:
import datasets
import tokenizers
tokenizers.__version__  # should be above or equal 0.10.0rc1

'0.13.2'

За функции предобработки текста, разделения его на токены и нумерикализации в 🤗 Tokenizers отвечает объект `Tokenizer`. Самый наглядный способ его создать - это предоставить ему словарь, мапящий слова на их индексы. Давайте напишем функцию, которая это делает.

**NOTE:** Самая простая операция токенизации - разделение строки на слова с помощью regex - стоит несколько сбоку в Tokenizers и даже не называется методом токенизации, тк зачастую она используется как первый шаг более сложного метода токенизации. Поэтому она задаётся с помощью объекта вида `pre_tokenizer`.

Для этого мы делаем функцию, которая в начале создаёт объект `pre_tokenizer` и препоцессит наши тексты. После чего мы подсчитываем частоты наших токенов и выбирает top-N самых частотных, это удобно делать с помощью объекта `Counter` из встроенного в Python модуля collections. В игрушечном примере в слещующей ячейке, состоящим только лишь из двух коротких предложений эта операция неважна, но когда вы работаете с текстами размерами с Википедию, ваш словарь быстро станет необъёмных размеров и его нужно уметь огрничивать.

Третий шаг - создание мапинга слов на их индексы. Он довольно простой и его можно сделать в одну строчку с помощью dictionary comprehension. И после этого мы готовы создать объект `Tokenizer` и присвоить ему претокенизатор, который мы использовали для разделения текста на слова.

In [3]:
from collections import Counter

import tokenizers
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace

texts = ['a list of sentences from my dataset', 'this is a text with known words']


def make_whitespace_tokenizer(texts, max_vocab_size=10_000, unk_token='UNK'):
    pre_tokenizer = Whitespace()
    tokenized_texts = [[w for w, _ in pre_tokenizer.pre_tokenize_str(t)] for t in texts]

    c = Counter()
    for text in tokenized_texts:
        c.update(text)

    token2id = {word: i + 1 for i, (word, count) in enumerate(c.most_common(max_vocab_size))}

    # usually, UNK is assigned index 0 or 1
    token2id[unk_token] = 0

    tokenizer = tokenizers.Tokenizer(WordLevel(token2id, unk_token))
    tokenizer.pre_tokenizer = pre_tokenizer
    return tokenizer


tokenizer = make_whitespace_tokenizer(texts)

# Encoding and decoding text

Для нумерикализации текста мы используем метод `.encode()`, возвращающий объект класса `Encoding`. Он содержит в себе много полезной информации, но самое главное из него - `.ids` дающий нам нумерикализованный текст, где слова заменены на их индексы.

In [4]:
_text = 'this is a text with unknown_word'
e = tokenizer.encode(_text)
e

Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [5]:
e.ids

[8, 9, 1, 10, 11, 0]

Денумерикализация делается методом `.decode()` (обратите внимание, что на вход к нему приходят `.ids`, а не объект `Encoding`). Эта операция очень важна на практике по многим причинам и одна из них - вы можете посмотреть на текст так, как его видит ваша нейросеть - с неизвестныи слоавми замененными на токен UNK и, возможно, с вамиши ошибками препроцессинга. Эти ошибки очень часты на практике и использование денумерикализации сильно помогает в дебаге.

Кроме этого нам не обойтись без этой операции в задачах генерации текста таких как машинный перевод, потому что на выходе из нашей модели мы ожидаем не последовательность чисел, а переведённый на другой язык текст.

In [6]:
tokenizer.decode(e.ids)

'this is a text with UNK'

Вы можете также использовать аттрибут `.tokens` у объекта `Encoding` и он выдаст вам список токенов в этом предложении. Обратите внимание, что в нём хранятся оригинальные токены текста, следовательно его не стоит использовать для дебага.

In [7]:
e.tokens

['this', 'is', 'a', 'text', 'with', 'UNK']

Ещё одна важная функция `Tokenizer` - это давать вам id одного слова или наоборот - выдавать слово по его id. Эти операции делаются методами `token_to_id` и `id_to_token`.

In [8]:
tokenizer.token_to_id('this')

8

In [9]:
tokenizer.id_to_token(8)

'this'

# Saving and loading the tokenizer

Если вы хотите задеплоить вашу обученную модель, вам нужны две вещи: ваша модель и пайплайн препроцессинга. `Tokenizer` по сути и является этим методом препроцессинга, тк он может включать в себя несколько шагов: нормализция и претокенизация, разбивание текста на токены и нумерикализация. Сохранение и загрузка `Tokenizer` реализована очень просто.

In [10]:
tokenizer.save('tokenizer.json')

Давайте посмотрим как выглядит этот файл

In [11]:
import json

with open('tokenizer.json') as f:
    tokenizer_json = json.load(f)

tokenizer_json

{'version': '1.0',
 'truncation': None,
 'padding': None,
 'added_tokens': [],
 'normalizer': None,
 'pre_tokenizer': {'type': 'Whitespace'},
 'post_processor': None,
 'decoder': None,
 'model': {'type': 'WordLevel',
  'vocab': {'UNK': 0,
   'a': 1,
   'list': 2,
   'of': 3,
   'sentences': 4,
   'from': 5,
   'my': 6,
   'dataset': 7,
   'this': 8,
   'is': 9,
   'text': 10,
   'with': 11,
   'known': 12,
   'words': 13},
  'unk_token': 'UNK'}}

Мы видим, что токенизатор сериализоват как простой и читаемый json и содержит в себе претокенизатор `Whitespace`, словарь, указание на то, какой токен используется как UNK-токен и многое другое о чём мы с вами поговорим в будущих занятиях.

Для загрузки токенизатора используется метод `from_file`.

In [12]:
loaded_tokenizer = tokenizers.Tokenizer.from_file('tokenizer.json')
e = loaded_tokenizer.encode('this is a text')
e

Encoding(num_tokens=4, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [13]:
e.ids

[8, 9, 1, 10]

# Batching

Ещё одна важная фича 🤗 Tokenizers - это возможность энкодить и декодить тексты в параллельных трэдах. Это становится очень важно, тк CPU-bottlenecks (когда предобработка занимает сопоставимое время с forward pass вашей нейросети) очень часты в NLP и могут сильно замедлить вашу тренировку, тк GPU будет ждать предобработки и ничего не делать в это время.

Для того, чтобы выполнить энкодинг в параллель, вы можете использовать метод `encode_batch`. Эта функциональность становится очень важной, когда размер вашего батча близок к 64 или больше.

In [14]:
texts

['a list of sentences from my dataset', 'this is a text with known words']

In [15]:
batch = tokenizer.encode_batch(texts)
batch

[Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [16]:
batch[0].ids, batch[1].ids

([1, 2, 3, 4, 5, 6, 7], [8, 9, 1, 10, 11, 12, 13])

Если вы хотите больше познакомиться с 🤗 Tokenizers, мы рекомендуем вам почитать их [документацию](https://huggingface.co/docs/tokenizers/python/latest). 

In [17]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
!unzip wikitext-103-raw-v1.zip

--2023-01-23 19:05:11--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.81.27, 54.231.203.96, 52.217.34.62, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.81.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191984949 (183M) [application/zip]
Saving to: ‘wikitext-103-raw-v1.zip’

wikitext-103-raw-v1 100%[===================>] 183.09M  61.4MB/s    in 3.0s    

2023-01-23 19:05:14 (61.4 MB/s) - ‘wikitext-103-raw-v1.zip’ saved [191984949/191984949]

Archive:  wikitext-103-raw-v1.zip
   creating: wikitext-103-raw/
  inflating: wikitext-103-raw/wiki.test.raw  
  inflating: wikitext-103-raw/wiki.valid.raw  
  inflating: wikitext-103-raw/wiki.train.raw  


In [18]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

In [19]:
tokenizer = Tokenizer(BPE(unk_token='[UNK]'))

To train our tokenizer on the wikitext files, we will need to instantiate a [trainer]{.title-ref}, in this case a BpeTrainer

In [20]:
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

We can set the training arguments like vocab_size or min_frequency (here left at their default values of 30,000 and 0) but the most important part is to give the special_tokens we plan to use later on (they are not used at all during training) so that they get inserted in the vocabulary.

The order in which you write the special tokens list matters: here "[UNK]" will get the ID 0, "[CLS]" will get the ID 1 and so forth.

We could train our tokenizer right now, but it wouldn’t be optimal. Without a pre-tokenizer that will split our inputs into words, we might get tokens that overlap several words: for instance we could get an "it is" token since those two words often appear next to each other. Using a pre-tokenizer will ensure no token is bigger than a word returned by the pre-tokenizer. Here we want to train a subword BPE tokenizer, and we will use the easiest pre-tokenizer possible by splitting on whitespace.

In [21]:
from tokenizers.pre_tokenizers import Whitespace
tokenizer.pre_tokenizer = Whitespace()

In [22]:
files = [f'/content/wikitext-103-raw/wiki.{split}.raw' for split in ['test', 'train', 'valid']]
tokenizer.train(files, trainer)

 To save the tokenizer in one file that contains all its configuration and vocabulary, just use the Tokenizer.save method:

In [23]:
tokenizer.save('/content/tokenizer-wiki.json')

and you can reload your tokenizer from that file with the Tokenizer.from_file classmethod:

In [24]:
tokenizer = Tokenizer.from_file('/content/tokenizer-wiki.json')

Using the tokenizer
Now that we have trained a tokenizer, we can use it on any text we want with the Tokenizer.encode method:

In [25]:
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")

This applied the full pipeline of the tokenizer on the text, returning an Encoding object.

This Encoding object then has all the attributes you need for your deep learning model (or other). The tokens attribute contains the segmentation of your text in tokens:

In [26]:
print(output.tokens)

['Hello', ',', 'y', "'", 'all', '!', 'How', 'are', 'you', '[UNK]', '?']


Similarly, the ids attribute will contain the index of each of those tokens in the tokenizer’s vocabulary:

In [27]:
print(output.ids)

[27253, 16, 93, 11, 5097, 5, 7961, 5112, 6218, 0, 35]


An important feature of the 🤗 Tokenizers library is that it comes with full alignment tracking, meaning you can always get the part of your original sentence that corresponds to a given token. Those are stored in the offsets attribute of our Encoding object. For instance, let’s assume we would want to find back what caused the "[UNK]" token to appear, which is the token at index 9 in the list, we can just ask for the offset at the index:

In [28]:
print(output.offsets[9])

(26, 27)


and those are the indices that correspond to the emoji in the original sentence:

In [29]:
sentence = "Hello, y'all! How are you 😁 ?"
sentence[26:27]

'😁'

Post-processing

We might want our tokenizer to automatically add special tokens, like "[CLS]" or "[SEP]". To do this, we use a post-processor. TemplateProcessing is the most commonly used, you just have to specify a template for the processing of single sentences and pairs of sentences, along with the special tokens and their IDs.

When we built our tokenizer, we set "[CLS]" and "[SEP]" in positions 1 and 2 of our list of special tokens, so this should be their IDs. To double-check, we can use the Tokenizer.token_to_id method:

In [30]:
tokenizer.token_to_id('[SEP]')

2

Here is how we can set the post-processing to give us the traditional BERT inputs:

In [31]:
from tokenizers.processors import TemplateProcessing

tokenizer.post_processor = TemplateProcessing(
    single='[CLS] $A [SEP]',
    pair='[CLS] $A [SEP] $B:1 [SEP]:1',
    special_tokens=[
        ('[CLS]', tokenizer.token_to_id('[CLS]')),
        ('[SEP]', tokenizer.token_to_id('[SEP]'))
    ]
)

Let’s go over this snippet of code in more details. First we specify the template for single sentences: those should have the form "[CLS] \$A [SEP]" where \$A represents our sentence.

Then, we specify the template for sentence pairs, which should have the form "[CLS] \$A [SEP] \$B [SEP]" where \$A represents the first sentence and $B the second one. The :1 added in the template represent the type IDs we want for each part of our input: it defaults to 0 for everything (which is why we don’t have \$A:0) and here we set it to 1 for the tokens of the second sentence and the last "[SEP]" token.

Lastly, we specify the special tokens we used and their IDs in our tokenizer’s vocabulary.

To check out this worked properly, let’s try to encode the same sentence as before:

In [32]:
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")
print(output.tokens)

['[CLS]', 'Hello', ',', 'y', "'", 'all', '!', 'How', 'are', 'you', '[UNK]', '?', '[SEP]']


To check the results on a pair of sentences, we just pass the two sentences to Tokenizer.encode:

In [33]:
output = tokenizer.encode("Hello, y'all!", "How are you 😁 ?")
print(output.tokens)

['[CLS]', 'Hello', ',', 'y', "'", 'all', '!', '[SEP]', 'How', 'are', 'you', '[UNK]', '?', '[SEP]']


You can then check the type IDs attributed to each token is correct with

In [34]:
print(output.type_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]


If you save your tokenizer with Tokenizer.save, the post-processor will be saved along.

Encoding multiple sentences in a batch
To get the full speed of the 🤗 Tokenizers library, it’s best to process your texts by batches by using the Tokenizer.encode_batch method:

In [35]:
output = tokenizer.encode_batch(["Hello, y'all!", "How are you 😁 ?"])

The output is then a list of Encoding objects like the ones we saw before. You can process together as many texts as you like, as long as it fits in memory.

To process a batch of sentences pairs, pass two lists to the Tokenizer.encode_batch method: the list of sentences A and the list of sentences B:

In [36]:
output = tokenizer.encode_batch(
    [["Hello, y'all!", "How are you 😁 ?"], ["Hello to you too!", "I'm fine, thank you!"]]
)

When encoding multiple sentences, you can automatically pad the outputs to the longest sentence present by using Tokenizer.enable_padding, with the pad_token and its ID (which we can double-check the id for the padding token with Tokenizer.token_to_id like before):

In [37]:
tokenizer.enable_padding(pad_id=3, pad_token='[PAD]')

We can set the direction of the padding (defaults to the right) or a given length if we want to pad every sample to that specific number (here we leave it unset to pad to the size of the longest text).

In [38]:
output = tokenizer.encode_batch(["Hello, y'all!", "How are you 😁 ?"])
print(output[1].tokens)

['[CLS]', 'How', 'are', 'you', '[UNK]', '?', '[SEP]', '[PAD]']


In this case, the attention mask generated by the tokenizer takes the padding into account:

In [39]:
print(output[1].attention_mask)

[1, 1, 1, 1, 1, 1, 1, 0]


Pretrained

Using a pretrained tokenizer

You can load any tokenizer from the Hugging Face Hub as long as a tokenizer.json file is available in the repository.

In [40]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_pretrained('bert-base-uncased')

Importing a pretrained tokenizer from legacy vocabulary files
You can also import a pretrained tokenizer directly in, as long as you have its vocabulary file. For instance, here is how to import the classic pretrained BERT tokenizer:

In [43]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt

--2023-01-23 19:09:25--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.41.134, 54.231.169.72, 52.217.164.64, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.41.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231508 (226K) [text/plain]
Saving to: ‘bert-base-uncased-vocab.txt’

bert-base-uncased-v 100%[===================>] 226.08K  --.-KB/s    in 0.009s  

2023-01-23 19:09:25 (23.4 MB/s) - ‘bert-base-uncased-vocab.txt’ saved [231508/231508]



In [44]:
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer('bert-base-uncased-vocab.txt', lowercase=True)